In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import pandas as pd

driver = webdriver.Chrome()
driver.get('https://www.oyez.org/cases/2022')

elems = driver.find_elements(By.XPATH, "//a[@ng-href]")
cases = []
for elem in elems:
    try:
        link = elem.get_attribute("href")
        if link.startswith('https://www.oyez.org/cases/2022/'):
            cases.append(link)
    except:
        print("no link")

time.sleep(1)
print(cases)

oyezdf = pd.DataFrame(columns=['CaseNo', 'CaseTitle', 'Pertains', 'Justice', 'Vote', 'Appellant', 'Appellee'])

for case in cases:
    driver.get(case)
    time.sleep(1)

    caseno = case.replace("https://www.oyez.org/cases/2022/", "")
    print(caseno)
    
    title = driver.find_element(By.XPATH, "//main/div/div/div/h1").text
    print(title)

    try: 
        appellant = title.split(sep=" v. ")[0]
        appellee = title.split(sep=" v. ")[1]
    except: 
        appellant = title
        appellee = "none"

    buzzwords = ["Amendment", "Section", "Act", "Clause"]
    about = driver.find_element(By.XPATH, "//div[@ng-bind-html=\"case.question\"]").text
    about = about.split(sep=" ")

    def checkbefore(about, word, result):
        if result == "":
            result = word
        x = about.index(word)
        try: 
            before = about[x-1]
            if before[0].isupper() == True:
                resultout = before + " " + result
                return checkbefore(about, before, resultout)
            elif before.isnumeric() == True:
                resultout = before + " " + result
                return checkafter(about, before, resultout)
            elif before == "of":
                resultout = before + " " + result
                return checkafter(about, before, resultout)
            elif before == "the":
                resultout = before + " " + result
                return checkafter(about, before, resultout)
            else: 
                return result
        except: 
            return result

    def checkafter(about, word, result):
        x = about.index(word)
        try: 
            after = about[x+1]
            if after[0].isupper() == True:
                if result == "":
                    resultout = after
                    return checkafter(about, after, resultout)
                else: 
                    resultout = result + " " + after
                    return checkafter(about, after, resultout)
            elif after.isnumeric() == True:
                if result == "":
                    resultout = after
                    return checkafter(about, after, resultout)
                else: 
                    resultout = result + " " + after
                    return checkafter(about, after, resultout)
            elif after == "of":
                if result == "":
                    resultout = after
                    return checkafter(about, after, resultout)
                else: 
                    resultout = result + " " + after
                    return checkafter(about, after, resultout)
            elif after == "the":
                if result == "":
                    resultout = after
                    return checkafter(about, after, resultout)
                else: 
                    resultout = result + " " + after
                    return checkafter(about, after, resultout)
            else: 
                return result
        except: 
            return result
            
    pertains = []
    for word in buzzwords:
        if word in about: 
            inresult = ""
            before = checkbefore(about, word, inresult)
            after = checkafter(about, word, inresult)
            pertains.append((f"{before}" + " " + f"{after}").replace("?", ""))

    try: 
        majority = driver.find_element(By.XPATH, "//span[@ng-if=\"decision.winning_party\"]").text
    except:
        majority = "not yet decided"

    time.sleep(2)
    
    photos = driver.find_elements(By.XPATH, "//figure/div/img")
    for photo in photos: 
        alt = photo.get_attribute("alt") # Justice's Name
        opacity = photo.value_of_css_property('opacity') # Justice's Vote
        if opacity == '1':
            votegroup = "Majority"
            if majority.lower() in appellant.lower():
                vote = appellant
            else:
                vote = appellee
        else: 
            votegroup = "Minority"
            if majority.lower() in appellant.lower():
                vote = appellee
            else:
                vote = appellant
        tempdf = pd.DataFrame.from_dict(data = {'CaseNo': [caseno], 'CaseTitle': [title], 'Pertains': [pertains], 'Justice': [alt], 'Vote': [votegroup], 'Appellant': [appellant], 'Appellee': [appellee]})
        oyezdf = pd.concat([oyezdf, tempdf], ignore_index=True)
    time.sleep(2)

print(oyezdf)

time.sleep(5)

driver.quit()

no link
no link
['https://www.oyez.org/cases/2022/21-476', 'https://www.oyez.org/cases/2022/21-1043', 'https://www.oyez.org/cases/2022/21-1086', 'https://www.oyez.org/cases/2022/21-757', 'https://www.oyez.org/cases/2022/21-869', 'https://www.oyez.org/cases/2022/21-432', 'https://www.oyez.org/cases/2022/22-592', 'https://www.oyez.org/cases/2022/21-1484', 'https://www.oyez.org/cases/2022/21-86', 'https://www.oyez.org/cases/2022/21-908', 'https://www.oyez.org/cases/2022/22-506', 'https://www.oyez.org/cases/2022/21-1195', 'https://www.oyez.org/cases/2022/22-714', 'https://www.oyez.org/cases/2022/21-1170', 'https://www.oyez.org/cases/2022/22-105', 'https://www.oyez.org/cases/2022/22-138', 'https://www.oyez.org/cases/2022/21-846', 'https://www.oyez.org/cases/2022/22O145', 'https://www.oyez.org/cases/2022/22-535', 'https://www.oyez.org/cases/2022/22-51', 'https://www.oyez.org/cases/2022/22-10', 'https://www.oyez.org/cases/2022/22-210', 'https://www.oyez.org/cases/2022/22-96', 'https://www.oye

: 

: 

In [ ]:
oyezdf

,CaseNo,CaseTitle,Pertains,Justice,Vote,Appellant,Appellee
0,21-1086,Allen v. Milligan,Act,"John G. Roberts, Jr.",Majority,Allen,Milligan
1,21-1086,Allen v. Milligan,Act,Clarence Thomas,Minority,Allen,Milligan
2,21-1086,Allen v. Milligan,Act,"Samuel A. Alito, Jr.",Minority,Allen,Milligan
3,21-1086,Allen v. Milligan,Act,Sonia Sotomayor,Majority,Allen,Milligan
4,21-1086,Allen v. Milligan,Act,Elena Kagan,Majority,Allen,Milligan
...,...,...,...,...,...,...,...
336,21-1164,Wilkins v. United States,Act,Amy Coney Barrett,Majority,Wilkins,United States
337,21-1164,Wilkins v. United States,Act,Ketanji Brown Jackson,Majority,Wilkins,United States
338,21-1164,Wilkins v. United States,Act,Clarence Thomas,Minority,Wilkins,United States
339,21-1164,Wilkins v. United States,Act,"John G. Roberts, Jr.",Minority,Wilkins,United States


In [ ]:
pd.DataFrame.to_csv(oyezdf, "oyezdf.csv")